## Constants

In [78]:
params = {
    'ema_short_period': 75,
    'ema_long_period': 100,
    'rsi_period': 14,
    'rsi_overbought_threshold': 60,
    'rsi_oversold_threshold': 40,
    'ema_crossunder_result_column': 'ema_crossunder',
    'ema_crossover_result_column': 'ema_crossover',
    'rsi_result_column': 'rsi',
    'ema_short_result_column': 'ema_short',
    'ema_long_result_column': 'ema_long',
    'ema_timeframe': '1d',
    'rsi_timeframe': '4h',
    'ema_cross_lookback_window': 7 # lookback window for EMA crossover of 7 days
}
SYMBOL = "BTC/EUR"
MARKET = "BITVAVO"

In [79]:
import importlib
import strategy as strategy_module
# Reload the strategy module to ensure it reflects the latest changes
importlib.reload(strategy_module)
Strategy = strategy_module.EMACrossoverRSIFFilterStrategy

In [80]:
from datetime import datetime, timezone
from investing_algorithm_framework import BacktestDateRange

visualization_date_range = BacktestDateRange(
    start_date=datetime(2023, 1, 1, tzinfo=timezone.utc),
    end_date=datetime(2024, 12, 31, tzinfo=timezone.utc)
)

In [81]:
from pathlib import Path
from investing_algorithm_framework import download

storage_path = Path.cwd().parent.parent / "resources" / "data" / "ohlcv"
rsi_data = download(
    symbol=SYMBOL,
    market=MARKET,
    time_frame=params["rsi_timeframe"],
    data_type="ohlcv",
    start_date=visualization_date_range.start_date,
    end_date=visualization_date_range.end_date,
    save=True,
    storage_path=str(storage_path)
)
ema_data = download(
    symbol=SYMBOL,
    market=MARKET,
    time_frame=params["ema_timeframe"],
    data_type="ohlcv",
    start_date=visualization_date_range.start_date,
    end_date=visualization_date_range.end_date,
    save=True,
    storage_path=str(storage_path)
)
close_data = download(
    symbol=SYMBOL,
    market=MARKET,
    time_frame=params["ema_timeframe"],
    data_type="ohlcv",
    start_date=visualization_date_range.start_date,
    end_date=visualization_date_range.end_date,
    save=True,
    storage_path=str(storage_path)
)

In [73]:
import pandas as pd
import plotly.graph_objects as go

def create_line_chart(
    data: pd.DataFrame,
    column,
    color="blue",
    name=""
):
    plot = go.Scatter(
        x=data.index,
        y=data[column],
        mode="lines",
        name=name,
        line=dict(color=color)
    )
    return plot


def create_crossover_graph(
    data: pd.DataFrame,
    column: str,
    lookup_column: str,
    color: str = "green",
    name: str = ""
):
    """
    Plot markers for crossover/crossunder signals.

    Args:
        data (pd.DataFrame): DataFrame with OHLCV + signal column.
        column (str): Name of the signal column (1 = event).
        lookup_column (str): Column to plot the y-values (e.g. "close").
        color (str): Marker color.
        name (str): Legend label.
    """
    signals = data[data[column] == 1]

    return {
        "x": signals.index,
        "y": signals[lookup_column],
        "mode": "markers",
        "marker": dict(
            color=color,
            size=10,
            symbol="triangle-up" if color == "green" else "triangle-down"
        ),
        "name": name,
    }

def create_crossunder_graph(
    data: pd.DataFrame,
    column: str,
    lookup_column: str,
    color: str = "red",
    name: str = ""
):
    """
    Plot markers for crossover/crossunder signals.

    Args:
        data (pd.DataFrame): DataFrame with OHLCV + signal column.
        column (str): Name of the signal column (1 = event).
        lookup_column (str): Column to plot the y-values (e.g. "close").
        color (str): Marker color.
        name (str): Legend label.
    """
    signals = data[data[column] == 1]

    return {
        "x": signals.index,
        "y": signals[lookup_column],
        "mode": "markers",
        "marker": dict(
            color=color,
            size=10,
            symbol="triangle-up" if color == "green" else "triangle-down"
        ),
        "name": name,
    }



def create_entries_exits(price, entries, exits):
    """
    Creates entry and exit markers for a Plotly figure.

    Args:
        entries (pd.Series): Series containing entry signals.
        exits (pd.Series): Series containing exit signals.

    Returns:
        tuple: A tuple containing the entry and exit scatter plots.
    """
    # Ensure alignment and boolean dtype
    entries = entries.reindex(price.index).fillna(False).astype(bool)
    exits   = exits.reindex(price.index).fillna(False).astype(bool)

    # Get coordinates where signals are True
    buy_x = entries[entries].index
    buy_y = price.loc[buy_x]

    sell_x = exits[exits].index
    sell_y = price.loc[sell_x]

    buy_trace = go.Scatter(
        x=buy_x,
        y=buy_y,
        mode='markers',
        marker=dict(symbol='triangle-up', size=10, color='green'),
        name='Buy Signal'
    )

    sell_trace = go.Scatter(
        x=sell_x,
        y=sell_y,
        mode='markers',
        marker=dict(symbol='triangle-down', size=10, color='red'),
        name='Sell Signal'
    )

    return buy_trace, sell_trace

def create_histogram_chart(
    data: pd.DataFrame,
    column,
    name=""
):
    # Histogram Bars (Enhanced)
    return go.Bar(
        x=data.index,
        y=data[column],
        name=name,
        marker=dict(
            color=['green' if v > 0 else 'red' for v in data[column]],
        ),
    )


def create_horizontal_line_chart(
    reference_x_data, value, color="black", name=None, dash="dash"
):
    x_idx = reference_x_data.index if hasattr(reference_x_data, "index") else pd.Index(reference_x_data)
    return go.Scatter(
        x=[x_idx.min(), x_idx.max()],
        y=[value, value],
        mode="lines",
        line=dict(color=color, width=2, dash=dash),
        name=name or f"y={value}",
    )

In [82]:

rsi_data_copy = rsi_data.copy()
close_data_copy = close_data.copy()
ema_data_copy = ema_data.copy()

strategy = Strategy(
    rsi_time_frame=params["rsi_timeframe"],
    rsi_period=params["rsi_period"],
    rsi_overbought_threshold=params["rsi_overbought_threshold"],
    rsi_oversold_threshold=params["rsi_oversold_threshold"],
    ema_time_frame=params["ema_timeframe"],
    ema_short_period=params["ema_short_period"],
    ema_long_period=params["ema_long_period"],
    ema_cross_lookback_window=params["ema_cross_lookback_window"],
    ema_long_result_column=params["ema_long_result_column"],
    ema_short_result_column=params["ema_short_result_column"],
    ema_crossover_result_column=params["ema_crossover_result_column"],
    ema_crossunder_result_column=params["ema_crossunder_result_column"],
)
ema_data_ind, rsi_data_ind = strategy.prepare_indicators(
    rsi_data=rsi_data_copy,
    ema_data=ema_data_copy,
)

In [ ]:
from plotly.subplots import make_subplots
from pyindicators import ema, rsi, crossover, crossunder

rsi_timeframe = params["rsi_timeframe"]
ema_timeframe = params["ema_timeframe"]

rsi_data_identifier = f"rsi_data"
ema_data_identifier = f"ema_data"

data = {
    rsi_data_identifier: rsi_data.copy(),
    ema_data_identifier: ema_data.copy()
}
entries = strategy.buy_signal_vectorized(data)
exits = strategy.sell_signal_vectorized(data)
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    subplot_titles=(
        "Entries/Exits",
        "EMA",
        "RSI"
    ),
    vertical_spacing=0.03,  # <-- smaller spacing between subplots
    row_heights=[0.2, 0.2, 0.2]  # optional: uniform row heights
)

# Entries chart
fig.add_trace(
    create_line_chart(close_data, column="Close", color="blue"),
    row=1,
    col=1
)

# Create entry and exit markers
entries, exits = create_entries_exits(close_data["Close"], entries, exits)

# Add entry and exit markers to the figure
fig.add_trace(entries, row=1, col=1)
fig.add_trace(exits, row=1, col=1)

fig.add_trace(
    create_line_chart(
        close_data,
        column="Close",
        color="blue"
    ),
    row=2,
    col=1
)

# Ema short chart
fig.add_trace(
    create_line_chart(
        ema_data_ind,
        column=params["ema_short_result_column"],
        color="green"
    ),
    row=2,
    col=1
)

# Ema long chart
fig.add_trace(
    create_line_chart(
        ema_data_ind,
        column=params["ema_long_result_column"],
        color="orange"
    ),
    row=2,
    col=1
)

# Crossover chart
fig.add_trace(
    create_crossover_graph(
        ema_data_ind,
        column=params["ema_crossover_result_column"],
        lookup_column="Close",
        color="green"
    ),
    row=2,
    col=1
)

# Crossunder chart
fig.add_trace(
    create_crossunder_graph(
        ema_data_ind,
        column=params["ema_crossunder_result_column"],
        lookup_column="Close",
        color="red"
    ),
    row=2,
    col=1
)

fig.add_trace(
    create_line_chart(
        rsi_data_ind,
        column=params["rsi_result_column"],
        color="purple"
    ),
    row=3,
    col=1
)

fig.add_trace(
    create_horizontal_line_chart(
        rsi_data_ind,
        value=params["rsi_overbought_threshold"],
        color="red",
        name="RSI Overbought"
    ),
    row=3,
    col=1
)

fig.add_trace(
    create_horizontal_line_chart(
        rsi_data_ind,
        value=params["rsi_oversold_threshold"],
        color="green",
        name="RSI Oversold"
    ),
    row=3,
    col=1
)
fig.update_layout(
    height=2000,   # make it taller (try 1000–1500 depending on how much space you want)
    width=1200,    # optional, make it wider
    title_text="Strategy Visualization",
    showlegend=False
)
fig.show()